Muzhou Liu muzhouliu@wustl.edu 465729

Yidi Zhang yidi.zhang@wustl.edu 465621

Ria Das  ria.das@wustl.edu 459319

Selamawit Tegegn sntegegn@wustl.edu 451841

In [1]:
import numpy as np
import pandas as pd 
# import tensorflow as tf 
import sklearn as sk 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

#from scipy import stats
#from scipy.stats import norm, skew 
from sklearn.gaussian_process.kernels import RBF
from sklearn.preprocessing import PolynomialFeatures

from sklearn.decomposition import PCA
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt  # Matlab-style plotting
#import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer

In [2]:
dataset_train = pd.read_csv('../input/train.csv')
print(dataset_train.head(3))
X = dataset_train.iloc[:, 1:12]
n = X.shape[0]
# print(X.head(3))
Y = dataset_train.iloc[:, 12]
# test train split
# # =============================================================================
dataset_test = pd.read_csv('../input/test.csv')
Xtest = dataset_test.iloc[:, 1:12]

X_all = pd.concat([X,Xtest], axis=0)

X_all['Soil_Type'] = X_all['Soil_Type'].astype('category') 
X_all = pd.get_dummies(X_all)



scaler = StandardScaler().fit(np.array(X.iloc[:,[0,1,2,3,4,5,6,7,8,9]]))
X_std = scaler.transform(np.array(X.iloc[:,[0,1,2,3,4,5,6,7,8,9]]))
X_std = pd.DataFrame(X_std)

X_std_full = pd.concat([X_std,X_all.iloc[:n,10::]],axis = 1)
print(list(X_std_full))

           ID  Elevation          ...           Soil_Type  From_Cache_la_Poudre
0   452309010       2468          ...                2703                     0
1   754045784       2052          ...                2717                     1
2  1347023875       2570          ...                2705                     0

[3 rows x 13 columns]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 'Soil_Type_2702', 'Soil_Type_2703', 'Soil_Type_2704', 'Soil_Type_2705', 'Soil_Type_2706', 'Soil_Type_2717', 'Soil_Type_4703', 'Soil_Type_4704', 'Soil_Type_4758', 'Soil_Type_5101', 'Soil_Type_6101', 'Soil_Type_6102', 'Soil_Type_7102', 'Soil_Type_7756', 'Soil_Type_7757']


/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [3]:
from sklearn.gaussian_process import GaussianProcessClassifier 
from sklearn.gaussian_process.kernels import RBF 

def sklearn_gpc(xtr, ytr, xte):
	kernel = 1.0*RBF(1.0)
	gpc = GaussianProcessClassifier(kernel= kernel, random_state =666, n_jobs =-1)

	gpc.fit(xtr, ytr)

	predict = gpc.predict_proba(xte)[:,1]
	# predict = gpc.predict(xte)

	return(predict)

In [4]:
dataset_test = pd.read_csv('../input/test.csv')
print(dataset_train.head(3))
Xtest = dataset_test.iloc[:, 1:12]
# print(Xtest.head(3))
Xtest['Soil_Type'] = Xtest['Soil_Type'].astype('category') 

scaler = StandardScaler().fit(np.array(X.iloc[:,[0,1,2,3,4,5,6,7,8,9]]))
Xtest_std = scaler.transform(np.array(Xtest.iloc[:,[0,1,2,3,4,5,6,7,8,9]]))
Xtest_std = pd.DataFrame(Xtest_std)
Xtest_std_full = pd.concat([Xtest_std,X_all.iloc[n::,10::]],axis = 1)



ids = dataset_test['ID']
predictions = sklearn_gpc(X_std_full,Y,Xtest_std_full)

output = pd.DataFrame({ 'ID' : ids, 'From_Cache_la_Poudre': predictions })
output.to_csv('svm_submission.csv', index = False)

           ID  Elevation          ...           Soil_Type  From_Cache_la_Poudre
0   452309010       2468          ...                2703                     0
1   754045784       2052          ...                2717                     1
2  1347023875       2570          ...                2705                     0

[3 rows x 13 columns]


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
